# Resource Maker

- creates resources using fhir.resources library
     1. load a skeleton yaml instance of a resource of type r_type
     2. create pymodel using costruct method
     3. update resource with elements defined elswhere like in a csv file based on:
         - element name
         - value
         
*step three is a work in progress*


### Set the current working directory

In [9]:
%cd '/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker'

/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker


### Import modules

In [22]:
from pathlib import Path
from fhir.resources import construct_fhir_element
from utils import new_dict, update_obj, validate
from yaml import load, FullLoader
from datetime import datetime, date, timezone, timedelta

### Provide test data

(need to start doing unit testing !!!)

In [26]:
timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
f_now = datetime.now(tzinfo)

class Row:
    def __init__(self):
        self.device_fhirid = 'Device/123'
        self.device_identifier = 'dev456'
        self.device_display = 'My Measuring Device'
        self.gateway_fhirid = 'Device/789'
        self.gateway_identifier = 'gateway2000'
        self.gateway_display = 'My Phone App'  
row = Row()

row.gateway_fhirid, f_now

('Device/789',
 datetime.datetime(2021, 7, 8, 10, 19, 27, 740794, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))))

### Get skeleton resource to create resource model

this is a short cut to get started instead of building from the ground up.

In [27]:
none_list = [None,{}]
r_type = 'Provenance'
#/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker/Provenance.yml
path = Path.cwd() /  f'{r_type}.yml'
r_dict = load(path.read_text(), Loader=FullLoader)
r_obj = construct_fhir_element(r_type,r_dict)
print(r_obj.yaml())

resourceType: Provenance
target:
- reference: Observation/123
recorded: 2019-07-09 15:26:23.217000+00:00
agent:
- type:
    coding:
    - system: http://terminology.hl7.org/CodeSystem/provenance-participant-type
      code: author
      display: Author
  who:
    reference: Device/123



### update the effective dataTime

In [28]:
r_obj.recorded = f_now
print(r_obj.yaml())

resourceType: Provenance
target:
- reference: Observation/123
recorded: 2021-07-08 10:19:27.740794-08:00
agent:
- type:
    coding:
    - system: http://terminology.hl7.org/CodeSystem/provenance-participant-type
      code: author
      display: Author
  who:
    reference: Device/123



### Add the device as Author

In [29]:
element = 'who'
element_dict = new_dict(
            reference = row.device_fhirid,
            identifier = new_dict(value = row.device_identifier),  # business identifier
            display = row.device_display,
            )
update_obj(element,element_dict,r_obj.agent[0])
print(r_obj.yaml())

resourceType: Provenance
target:
- reference: Observation/123
recorded: 2021-07-08 10:19:27.740794-08:00
agent:
- type:
    coding:
    - system: http://terminology.hl7.org/CodeSystem/provenance-participant-type
      code: author
      display: Author
  who:
    reference: Device/123
    identifier:
      value: dev456
    display: My Measuring Device



### Add the gateway as transmitter

use transmitter for now and consider other types later

In [30]:
element = 'agent'
agent_type_system = "http://hl7.org/fhir/us/core/CodeSystem/us-core-provenance-participant-type"
agent_type_code = "transmitter"
agent_type_display = "Transmitter"

who_dict = new_dict(
            reference = row.gateway_fhirid,
            identifier = new_dict(value = row.gateway_identifier),  # business identifier
            display = row.gateway_display,
            )

coding_dict = new_dict(
            system = agent_type_system,
            code = agent_type_code,
            display = agent_type_display,
            )

element_dict =  new_dict(
      type = new_dict(coding = [coding_dict],
             text='Transmitter'),
      who = who_dict,
    )

update_obj(element,element_dict,r_obj)
print(r_obj.yaml())

resourceType: Provenance
target:
- reference: Observation/123
recorded: 2021-07-08 10:19:27.740794-08:00
agent:
- type:
    coding:
    - system: http://terminology.hl7.org/CodeSystem/provenance-participant-type
      code: author
      display: Author
  who:
    reference: Device/123
    identifier:
      value: dev456
    display: My Measuring Device
- type:
    coding:
    - system: http://hl7.org/fhir/us/core/CodeSystem/us-core-provenance-participant-type
      code: transmitter
      display: Transmitter
    text: Transmitter
  who:
    reference: Device/789
    identifier:
      value: gateway2000
    display: My Phone App



### Validate

In [31]:
validate(r_obj)

foo
Provenance
...validating


### Save Locally

In [32]:
ig_source_path = f'examples/{r_type}'
path = Path.cwd() / ig_source_path / f'example.yaml' 
print(f'...........saving to file {path}............')
path.write_text(r_obj.yaml())

...........saving to file /Users/ehaas/Documents/Python/MyNotebooks/Obs_maker/examples/Provenance/example.yaml............


637